<a href="https://colab.research.google.com/github/baoquan25/Machine_learning_bigquery/blob/main/Machine_learning_in_BIGQUERY.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
### Lưu ý: Code chỉ sử dụng đước trên BIGQUERY
CREATE OR REPLACE MODEL `my-project-11102022-438501.DA04.trans_model`
OPTIONS(MODEL_TYPE = 'LOGISTIC_REG',
  INPUT_LABEL_COLS = ['Label'],
  AUTO_CLASS_WEIGHTS = TRUE)
AS
SELECT
  IF(totals.transactions IS NULL,0,1) AS Label,
  IFNULL(device.operatingSystem,"") AS OS,
  device.isMobile AS isMobile,
  IFNULL(geoNetwork.country,"") AS Country,
  IFNULL(totals.pageviews,0) AS Pageviews
FROM `bigquery-public-data.google_analytics_sample.ga_sessions_*`
WHERE
  _TABLE_SUFFIX BETWEEN '20160801' AND '20170630' AND RAND()<0.01
;

# Danh gia model, dung du lieu test, truy van tuong tu nhu train, chi doi nguon du lieu

SELECT *
FROM ML.EVALUATE(MODEL `my-project-11102022-438501.DA04.trans_model`,(
 SELECT
  IF(totals.transactions IS NULL,0,1) AS Label,
  IFNULL(device.operatingSystem,"") AS OS,
  device.isMobile AS isMobile,
  IFNULL(geoNetwork.country,"") AS Country,
  IFNULL(totals.pageviews,0) AS Pageviews
FROM `bigquery-public-data.google_analytics_sample.ga_sessions_*`
WHERE
  _TABLE_SUFFIX BETWEEN '20170701' AND '20170731'
));

#du doan moi
SELECT *
FROM ML.PREDICT(MODEL `my-project-11102022-438501.DA04.trans_model`,(
 SELECT
  IF(totals.transactions IS NULL,0,1) AS Label,
  IFNULL(device.operatingSystem,"") AS OS,
  device.isMobile AS isMobile,
  IFNULL(geoNetwork.country,"") AS Country,
  IFNULL(totals.pageviews,0) AS Pageviews
FROM `bigquery-public-data.google_analytics_sample.ga_sessions_20170801`
))
WHERE
  RAND()<0.01
LIMIT 10;

In [ ]:
## Show data
Select *
from `my-project-11102022-438501.DA04.electric`
where extract (year from DATE_) >= 1985
order by DATE_
limit 5

## build model
CREATE OR REPLACE MODEL DA04.electric_arima_model
OPTIONS
(model_type = 'ARIMA_PLUS',
  time_series_timestamp_col = 'DATE_',
  time_series_data_col = 'IPG2211A2N'
) AS
Select *
from `my-project-11102022-438501.DA04.electric`
where extract (year from DATE_) >= 1985
order by DATE_

## EVALUATE
SELECT *
FROM ML.ARIMA_EVALUATE (MODEL DA04.electric_arima_model);

## PREDICTION
SELECT *
FROM ML.FORECAST(MODEL  DA04.electric_arima_model,
                STRUCT(12 AS HORIZON,0.8 AS CONFIDENCE_LEVEL))

In [ ]:
## lay du lieu  va kiem tra

SELECT count(*)
from `my-project-11102022-438501.DA04.weather`

SELECT *
from `my-project-11102022-438501.DA04.weather`
limit 3

CREATE OR REPLACE TABLE DA04.weather_new
AS
SELECT *,
 CASE
  WHEN split_field <= 0.8 THEN 'training'
  WHEN split_field > 0.8 THEN 'evaluation'
 END AS split_dataframe
FROM (
    ## tạo cột split_filed [0.001-> 1.000]
  SELECT * , ROUND(ABS(RAND()),3) AS split_field
  FROM `my-project-11102022-438501.DA04.weather_new`
);

SELECT split_dataframe, count(split_field) as num_rows
FROM `my-project-11102022-438501.DA04.weather_new`
GROUP BY split_dataframe;


CREATE OR REPLACE MODEL DA04.weather_linear_model
OPTIONS(
  model_type ='linear_reg',
  input_label_cols = ['Temperature_c']
)
AS
SELECT * EXCEPT (split_field,split_dataframe)
FROM `my-project-11102022-438501.DA04.weather_new`
WHERE Temperature_c IS NOT NULL AND split_dataframe like 'training'

#mean, std
SELECT
  AVG(Temperature_c) as mean,
  STDDEV(Temperature_c) as std_dev_sample,
FROM `my-project-11102022-438501.DA04.weather_new`
WHERE Temperature_c is not null
-- mean = 11.86, std = 9.51

SElECT *
FROM ML.EVALUATE(model DA04.weather_linear_model,(
SELECT * EXCEPT (split_field,split_dataframe)
FROM `my-project-11102022-438501.DA04.weather_new`
WHERE Temperature_c IS NOT NULL AND split_dataframe like 'evaluation'))

## Make new prediction
SELECT
*
FROM ML.PREDICT(MODEL DA04.weather_linear_model, (
  SELECT 0.79 as Humidity,
  14.3 as Wind_Speed_kmh,
  131 as Wind_Bearing_degrees,
  10.3 as Visibility_km,
  1010.15 as Pressure_millibars ,
  1 as Rain,
  'Normal' as Description
  )
);

## LOGISTIC REGRESSION
Create or replace model DA04.weather_logistic_model
options( model_type ='BOOSTED_TREE_REGRESSOR',
        INPUT_LABEL_COLS = ['Temperature_c']
) as
select *EXCEPT (split_field,split_dataframe)
FROM `my-project-11102022-438501.DA04.weather_new`

